<a href="https://colab.research.google.com/github/morim3/qiskit-optimization/blob/main/my_experiment/compare_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf qiskit-optimization/
!git clone https://github.com/morim3/qiskit-optimization

Cloning into 'qiskit-optimization'...
remote: Enumerating objects: 4195, done.
remote: Counting objects: 100% (260/260), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 4195 (delta 166), reused 166 (delta 88), pack-reused 3935
Receiving objects: 100% (4195/4195), 5.29 MiB | 10.23 MiB/s, done.
Resolving deltas: 100% (2779/2779), done.


In [2]:
!nvidia-smi
# !pip install -r qiskit-optimization/requirements-dev.txt
!pip install qiskit-aer-gpu
!pip install docplex
!pip install pylatexenc

import sys
sys.path.append("./qiskit-optimization/")

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 22.8 MB 1.6 MB/s 
     |████████████████████████████████| 4.8 MB 40.5 MB/s 
     |████████████████████████████████| 37.5 MB 156 kB/s 
     |████████████████████████████████| 929 kB 52.7 MB/s 
     |████████████████████████████████| 50 kB 3.1 MB/s 
     |████████████████████████████████| 49 kB 3.4 MB/s 
     |████████████████████████████████| 1.9 MB 41.5 MB/s 
     |████████████████████████████████| 112 kB 69.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 640 kB 7.4 MB/s 
  Created wheel for docplex: filename=docplex-2.24.232-py3-none-any.whl size=682306 sha256=d9a5582b21ab5f5dfe3dadb55b8d0e219ce0103c1ba52420b

In [3]:
%load_ext autoreload
%autoreload

from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit_optimization.algorithms import PQCGroverOptimizer, GroverOptimizer, MinimumEigenOptimizer
from qiskit_optimization.algorithms.pqc_grover_optimizer import ProposeDistribution
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp
from qiskit import Aer, QuantumCircuit
from qiskit.utils import QuantumInstance
from qiskit.circuit.library import TwoLocal
from docplex.mp.model import Model
import numpy as np
print(Aer.backends())
backend = Aer.get_backend('aer_simulator')
backend.set_options(device='GPU')
backend_statevector = Aer.get_backend('statevector_simulator')
print(backend)

[AerSimulator('aer_simulator'), AerSimulator('aer_simulator_statevector'), AerSimulator('aer_simulator_density_matrix'), AerSimulator('aer_simulator_stabilizer'), AerSimulator('aer_simulator_matrix_product_state'), AerSimulator('aer_simulator_extended_stabilizer'), AerSimulator('aer_simulator_unitary'), AerSimulator('aer_simulator_superop'), QasmSimulator('qasm_simulator'), StatevectorSimulator('statevector_simulator'), UnitarySimulator('unitary_simulator'), PulseSimulator('pulse_simulator')]
aer_simulator_gpu


In [4]:
def get_problem(dim):
  import itertools
  model = Model()
  xs = [model.binary_var(name=f"x{i}") for i in range(dim)]
  objective = sum([np.random.randint(1, 4) * x for x in xs]) + \
      sum([np.random.randint(-4, -1) * x * y 
      for x, y in itertools.combinations(xs, 2) 
      if np.random.choice([0, 1], p=[2/3, 1/3])==1])
  # objective = (-xs[0]+2*xs[1]-3*xs[2]-2*xs[0]*xs[2]-1*xs[1]*xs[2])
  model.minimize(objective)
  qp = from_docplex_mp(model)
  return qp

def eval_num(operation_dict, optimal_iteration):
    eval_num = 0
    for i, x in enumerate(operation_dict.values()):
      if i <= optimal_iteration:
        eval_num += x['Q(x)']
        if 'Q' in x:
            eval_num += x['Q']
    return eval_num

def exact_solution(qp):
  exact_solver = MinimumEigenOptimizer(NumPyMinimumEigensolver())
  exact_result = exact_solver.solve(qp)
  return exact_result.fval

def pqc_optimizer(qp, exact_fval):
  num_key_qbits = len(qp.objective.linear.to_array())
  init_state = QuantumCircuit(num_key_qbits)
  init_state.h(range(num_key_qbits))
  ansatz = TwoLocal(num_key_qbits, 'ry', 'crx', 'circular', reps=1,
                    insert_barriers=True,
                    initial_state=init_state,
                    skip_final_rotation_layer=True,
                    )
  
  init_param = np.zeros(ansatz.num_parameters)
  propose_distribution = ProposeDistribution(
      ansatz,                 
      num_key_qbits, 
      QuantumInstance(backend=backend, shots=1024, seed_simulator=99),
      init_param=init_param
      )
  grover_optimizer = PQCGroverOptimizer(
      num_value_qubits=8, 
      propose_dist=propose_distribution,
      num_iterations=15, 
      quantum_instance=backend,      
      sample_num=1, quantile=0.,
      threshold_lr=1. ,
      positive_train_conf={"iter": 5, "learning_rate": 0.001},
      negative_train_conf={"iter": 5, "learning_rate": 0.0001})
  results = grover_optimizer.solve(qp)
  is_optimal = results.intermediate_fval <= exact_fval
  eval_num = eval_num(results.operation_counts, results._optimal_iteration)
  return is_optimal, eval_num

def pbil_optimizer(qp, exact_fval):
  grover_optimizer = PBILGroverOptimizer(8, num_iterations=15, quantum_instance=backend, 
                                       pbil_sample_num=1, quantile=0., lr=0.05, )
  results = grover_optimizer.solve(qp)

def grover_optimizer(qp, exact_fval):
  grover_optimizer = GroverOptimizer(8, num_iterations=15, quantum_instance=backend,)
  results = grover_optimizer.solve(qp)
  is_optimal = results.intermediate_fval <= exact_fval
  eval_num = eval_num(results.operation_counts, results._optimal_iteration)
  
  return is_optimal, eval_num

def pbil_optimizer(qp, exact_fval):
  grover_optimizer = PBILGroverOptimizer(8, num_iterations=15, quantum_instance=backend, 
                                       pbil_sample_num=1, quantile=0., lr=0.05, )
  results = grover_optimizer.solve(qp)
  is_optimal = results.intermediate_fval <= exact_fval
  eval_num = eval_num(results.operation_counts, results._optimal_iteration)
  return is_optimal, eval_num
  

def compere_performance(dim, exp_num):
  pqc_evals = []
  pbil_evals = []
  grover_evals = []
  for n_exp in range(exp_num):
    qp = get_problem(dim)
    exact_fval = exact_solution(qp)

    is_optimal, eval_num = pqc_optimizer(qp, exact_fval)
    if is_optimal:
      pqc_evals.append(eval_num)

    is_optimal, eval_num = pbil_optimizer(qp, exact_fval)
    if is_optimal:
      pbil_evals.append(eval_num)

    is_optimal, eval_num = grover_optimizer(qp, exact_fval)
    if is_optimal:
      grover_evals.append(eval_num)

  return np.array(pqc_evals), np.array(pbil_evals), np.array(grover_evals)



In [5]:
results = []
for dim in [5, 7, 10, 13]:
  results.append(compare_performance(dim, 5))

NameError: ignored